### Solução Execício 1 -  Tutoria de Machine Learning 2-2025

Utilizando esse conjunto de dados: https://www.kaggle.com/datasets/disham993/9000-movies-dataset

Responda as seguintes perguntas:

1. Qual tamanho do DataSet?
2. Quantas linhas?
3. Quantas colunas?
4. Qual o tipo de variável de cada coluna?
5. Qual o filme com maior número de votaçôes?
6. Qual filme teve a maior nota (critério de desempate é o filme com mais votos)
7. Existem valores nulos? Se sim, qual tratamento irá realizar? (Se não temos nome de algum filme, melhor nem considerar)
8. Transforme as variaveis categóricas de linguagem e genero em númericas (utilize dummy)
9. Normalize as variaveis numéricas
10. Armazene esses valores como um artefato dentro do MLFlow
11. Quais insights é possivel obter desses dados?

In [47]:
# Importando Bibliotecas
import pandas as pd
import numpy as np
# Bibliotecas para aprendizado de máquina
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import mlflow

In [48]:
#Carregando dados
df = pd.read_csv('mymoviedb.csv', engine='python')
df.head()

,Release_Date,Title,Overview,Popularity,Vote_Count,Vote_Average,Original_Language,Genre,Poster_Url
0,2021-12-15,Spider-Man: No Way Home,Peter Parker is unmasked and no longer able to...,5083.954,8940,8.3,en,"Action, Adventure, Science Fiction",https://image.tmdb.org/t/p/original/1g0dhYtq4i...
1,2022-03-01,The Batman,"In his second year of fighting crime, Batman u...",3827.658,1151,8.1,en,"Crime, Mystery, Thriller",https://image.tmdb.org/t/p/original/74xTEgt7R3...
2,2022-02-25,No Exit,Stranded at a rest stop in the mountains durin...,2618.087,122,6.3,en,Thriller,https://image.tmdb.org/t/p/original/vDHsLnOWKl...
3,2021-11-24,Encanto,"The tale of an extraordinary family, the Madri...",2402.201,5076,7.7,en,"Animation, Comedy, Family, Fantasy",https://image.tmdb.org/t/p/original/4j0PNHkMr5...
4,2021-12-22,The King's Man,As a collection of history's worst tyrants and...,1895.511,1793,7.0,en,"Action, Adventure, Thriller, War",https://image.tmdb.org/t/p/original/aq4Pwv5Xeu...


### 1. Qual tamanho do DataSet?

In [49]:

tamanho = df.shape
print(f'1. Qual tamanho do DataSet?\nO dataset possuí o tamanho de {df.size} elementos.')
print()
print(f'2. Quantas linhas?\nNúmero de linhas igual a {tamanho[0]}.')
print()
print(f'3. Quantas colunas?\nNúmero de colunas igula a {tamanho[1]}.')


1. Qual tamanho do DataSet?
O dataset possuí o tamanho de 88533 elementos.

2. Quantas linhas?
Número de linhas igual a 9837.

3. Quantas colunas?
Número de colunas igula a 9.


### 4. Qual o tipo de variável de cada coluna?

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9837 entries, 0 to 9836
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Release_Date       9837 non-null   object 
 1   Title              9828 non-null   object 
 2   Overview           9828 non-null   object 
 3   Popularity         9827 non-null   float64
 4   Vote_Count         9827 non-null   object 
 5   Vote_Average       9827 non-null   object 
 6   Original_Language  9827 non-null   object 
 7   Genre              9826 non-null   object 
 8   Poster_Url         9826 non-null   object 
dtypes: float64(1), object(8)
memory usage: 691.8+ KB


### 5. Qual o filme com maior número de votaçôes?

In [51]:
#Transformando valores da coluna Vote_Average de object para float
df['Popularity'] = pd.to_numeric(df['Popularity'], errors='coerce')
df['Vote_Count'] = pd.to_numeric(df['Vote_Count'], errors='coerce')
df['Vote_Average'] = pd.to_numeric(df['Vote_Average'], errors = 'coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9837 entries, 0 to 9836
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Release_Date       9837 non-null   object 
 1   Title              9828 non-null   object 
 2   Overview           9828 non-null   object 
 3   Popularity         9827 non-null   float64
 4   Vote_Count         9826 non-null   float64
 5   Vote_Average       9826 non-null   float64
 6   Original_Language  9827 non-null   object 
 7   Genre              9826 non-null   object 
 8   Poster_Url         9826 non-null   object 
dtypes: float64(3), object(6)
memory usage: 691.8+ KB


In [52]:
#Filtrando valores para o filme de maior número de votos

mais_votado = df['Vote_Count'].max()
nome = df.loc[df['Vote_Count']== mais_votado, 'Title']

# Exibindo Resultados
print(f'Votos: {mais_votado:.0f}')
print(f'Titulo: {nome.values[0]}')

Votos: 31077
Titulo: Inception


### 6. Qual filme teve a maior nota (critério de desempate é o filme com mais votos)

In [53]:


#Filtrando maior nota
maior_nota = df['Vote_Average'].max()

#Verificando se há mais de um filme com maior nota
empate = df[df['Vote_Average'] == maior_nota]

#Filme com maior nota e maior número de votos
vencedor = empate.loc[empate['Vote_Count'].idxmax()]

# Exibindo resultado
print(f"Título: {vencedor['Title']}")
print(f"Nota: {vencedor['Vote_Average']}")
print(f"Votos: {vencedor['Vote_Count']}")

Título: Kung Fu Master Huo Yuanjia
Nota: 10.0
Votos: 1.0


### 7. Existem valores nulos? Se sim, qual tratamento irá realizar? (Se não temos nome de algum filme, melhor nem considerar

In [54]:
df.isnull().sum()


Release_Date          0
Title                 9
Overview              9
Popularity           10
Vote_Count           11
Vote_Average         11
Original_Language    10
Genre                11
Poster_Url           11
dtype: int64

Primeiro tratamento para os valores vazios. 
Excluir as linhas da coluna Titulo que estão vazias, pois sem o nome do filme não é possível fazer analises.
Após remover verificar novamente os valores vazios das demais colunas.

In [55]:
df = df.dropna(subset=['Title'])
df.isnull().sum()

Release_Date         0
Title                0
Overview             0
Popularity           1
Vote_Count           2
Vote_Average         2
Original_Language    1
Genre                2
Poster_Url           2
dtype: int64

Os valores restantes em condição vazio podem ser tratadas da seguinte forma:

Overview: pode ficar vazia

Popularity: substituir por zero

Vote_Count: substituir por zero

Vote_Average: substituir pela média da coluna

Original_Lenguage: substituir por desconhecido "unknown"

Genre: substituir por desconhecido "unknown"

Post_Url: pode ficar vazia



In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9828 entries, 0 to 9836
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Release_Date       9828 non-null   object 
 1   Title              9828 non-null   object 
 2   Overview           9828 non-null   object 
 3   Popularity         9827 non-null   float64
 4   Vote_Count         9826 non-null   float64
 5   Vote_Average       9826 non-null   float64
 6   Original_Language  9827 non-null   object 
 7   Genre              9826 non-null   object 
 8   Poster_Url         9826 non-null   object 
dtypes: float64(3), object(6)
memory usage: 767.8+ KB


In [57]:
for col in ['Popularity', 'Vote_Count']:
    df[col] = df[col].fillna(0)

for col in ['Vote_Average']:
    df[col] = df[col].fillna(df[col].mode())

for col in ['Original_Language', 'Genre']:
    df[col] = df[col].fillna('unknow')



df.isnull().sum()

Release_Date         0
Title                0
Overview             0
Popularity           0
Vote_Count           0
Vote_Average         2
Original_Language    0
Genre                0
Poster_Url           2
dtype: int64

### 8. Transforme as variaveis categóricas de linguagem e genero em númericas (utilize dummy)

In [58]:
# Gerar as colunas one-hot
df_one_hot = pd.get_dummies(df[['Original_Language', 'Genre']], drop_first='False', dtype=int)

# Remover as colunas originais
df.drop(['Original_Language', 'Genre'], axis=1, inplace=True)

# Concatenar as novas colunas codificadas
df = pd.concat([df, df_one_hot], axis=1)
df.shape


(9828, 2388)

### 9. Normalize as variaveis numéricas

In [59]:
df_minimax = df.select_dtypes(include=[np.number]).drop(columns=['Vote_Average'])
minimax = MinMaxScaler()
df_minimax = pd.DataFrame(minimax.fit_transform(df_minimax), columns=df_minimax.columns)
print('Visualização após Normalização (Min-Max):')
display(df_minimax.describe().T.head())


Visualização após Normalização (Min-Max):


,count,mean,std,min,25%,50%,75%,max
Popularity,9828.0,0.007930,0.021414,0.0,0.003172,0.004168,0.006918,1.0
Vote_Count,9828.0,0.044813,0.084021,0.0,0.004698,0.014287,0.044277,1.0
Original_Language_bn,9828.0,0.000102,0.010087,0.0,0.000000,0.000000,0.000000,1.0
Original_Language_ca,9828.0,0.000102,0.010087,0.0,0.000000,0.000000,0.000000,1.0
Original_Language_cn,9828.0,0.011091,0.104732,0.0,0.000000,0.000000,0.000000,1.0


In [60]:
df_standard = df.select_dtypes(include=[np.number]).drop(columns=['Vote_Average'])
standard = StandardScaler()
df_standard = pd.DataFrame(standard.fit_transform(df_standard), columns=df_standard.columns)
print('Visualizando após Normalização (StandadSacaler)')
display(df_standard.describe().T.head())

Visualizando após Normalização (StandadSacaler)


,count,mean,std,min,25%,50%,75%,max
Popularity,9828.0,-4.712348e-16,1.000051,-0.370338,-0.222204,-0.175696,-0.047277,46.329715
Vote_Count,9828.0,-1.028815e-15,1.000051,-0.533386,-0.477468,-0.363335,-0.006381,11.369038
Original_Language_bn,9828.0,6.199515e-16,1.000051,-0.010088,-0.010088,-0.010088,-0.010088,99.131226
Original_Language_ca,9828.0,3.820797e-16,1.000051,-0.010088,-0.010088,-0.010088,-0.010088,99.131226
Original_Language_cn,9828.0,-3.624082e-15,1.000051,-0.105902,-0.105902,-0.105902,-0.105902,9.442729


In [61]:
# Escolhendo MiniMaxScaler como técnica
# Renomear dataset final
dados_processados = df_minimax.copy()

# Salvar
processed_data_path = "dados_processados.csv"
dados_processados.to_csv(processed_data_path, index=False)
print(f"dados_processados salvo: {processed_data_path} | shape: {dados_processados.shape}")

dados_processados salvo: dados_processados.csv | shape: (9828, 2383)


### Armazenamento e Versionamento de Features com MLflow

In [62]:
# Salvar o dataset processado como um arquivo CSV
processed_data_path = "dados_processados.csv"
dados_processados.to_csv(processed_data_path, index=False)
print("Dataset processado salvo localmente.")

# Registrar o dataset processado como artefato no MLflow
mlflow.start_run()  # Iniciar um novo experimento
mlflow.log_artifact(processed_data_path)  # Registrar o arquivo como artefato
mlflow.end_run()  # Encerrar o experimento

print("Features armazenadas e versionadas com sucesso no MLflow!")

Dataset processado salvo localmente.
Features armazenadas e versionadas com sucesso no MLflow!
